In [53]:
import pandas as pd
import requests
import json
from simplejson import JSONDecodeError
import warnings
import psycopg2
import yaml

In [54]:
airports_spain = pd.read_csv('./data/airports_spain.csv')[['ICAO', 'City']]

In [55]:
#https://avwx.rest/

with open('./keys/keys.yml') as file:
    keys = yaml.load(file)

headers = {
  'Authorization': keys['key1'],
  'Content-Type': 'application/json'
}

headers2 = {
  'Authorization': keys['key2'],
  'Content-Type': 'application/json'
}

reporting = "true"
formatting = "json"
onfail = "error"

In [56]:
conn = psycopg2.connect(
    host=keys['pghost'],
    database="metar",
    port=35728,
    user=keys['pguser'],
    password=keys['pgpass'])

In [57]:
def parseo (url, city):
    
    #Request
    response = requests.get( url, headers = headers, verify=False )
    if response.status_code == 400 or response.status_code == 204:
        return None
    else:
        json_response = json.loads(response.text)

        #Parsing
        metar = json_response['raw']
        station = json_response['station']
        pressure = json_response['altimeter']['value']
        pressure_unit = json_response['units']['altimeter']
        wind_direction = json_response['wind_direction']['value']
        wind_unit = 'º'
        wind_speed = json_response['wind_speed']['value'] 
        wind_speed_unit = json_response['units']['wind_speed']
        visibility_str = json_response['visibility']['repr']
        visibility_number = json_response['visibility']['value']
        visibility_unit = json_response['units']['visibility']
        temperature = json_response['temperature']['value'] 
        temperature_unit = json_response['units']['temperature']
        remarks = json_response['remarks']
        flight_rules = json_response['flight_rules']
        time = json_response['time']['dt']
        identificadorUnico = str(time) + station

        clouds = []
        if not json_response['clouds']:
            clouds = ["No clouds"]
        else:
            for element in json_response['clouds']:
                clouds.append(element['repr'])
        if json_response['wind_gust'] == None:
            json_response['wind_gust'] = 0
        wind_gust = json_response['wind_gust']
        wind_gust_unit = json_response['units']['wind_speed']
        try: 
            wind_direction_var_1 = json_response['wind_variable_direction'][0]['value']
            wind_direction_var_1_unit = 'º'
        except:
            wind_direction_var_1 = None
            wind_direction_var_1_unit = None
        try: 
            wind_direction_var_2 = json_response['wind_variable_direction'][1]['value']
            wind_direction_var_2_unit = 'º'
        except:
            wind_direction_var_2 = None
            wind_direction_var_2_unit = None

        #Storing
        readyforinsertion = (identificadorUnico, metar, time, station, city, pressure, pressure_unit, clouds, wind_direction, wind_unit, wind_speed, wind_speed_unit, wind_direction_var_1, wind_direction_var_1_unit, 
                             wind_direction_var_2, wind_direction_var_2_unit, visibility_str, visibility_number, visibility_unit, temperature, temperature_unit, remarks, flight_rules)
        return readyforinsertion

In [58]:
with open('./sql.yml') as file:
    sql_sentences = yaml.load(file)

try:
    cur = conn.cursor()
    cur.execute(sql_sentences['create_table'])
    conn.commit()
    cur.close()
except:
    print("Rollback")
    conn.rollback()

In [59]:
warnings.simplefilter("ignore")

for airport in airports_spain.itertuples():
    url = "https://avwx.rest/api/metar/" + airport[1] + "?reporting=" + reporting + "&format=" + formatting + "&onfail=" + onfail
    data = parseo(url, airport[2])
    if data == None:
        print('Airport ' + airport[1] + ' data not found')
        continue
    else:
        print(data)
        try:
            cur = conn.cursor()
            cur.execute(sql_sentences['insert_table'], data)
            conn.commit()
            print("Insertada")
            cur.close()
        except Exception as e:
            print("Rollback")
            print(e)
            conn.rollback()

('GCFV 101000Z 04016KT 9999 BKN022 25/19 Q1014', '2020-08-10T10:00:00Z', 'GCFV', 'Fuerteventura', 1014, 'hPa', ['BKN022'], 40, 'º', 16, 'kt', None, None, None, None, '9999', 9999, 'm', 25, 'C', '', 'MVFR')
Insertada
('GCHI 101000Z 36013KT 340V040 9999 FEW012 SCT021 24/21 Q1016', '2020-08-10T10:00:00Z', 'GCHI', 'Hierro', 1016, 'hPa', ['FEW012', 'SCT021'], 360, 'º', 13, 'kt', 340, 'º', 40, 'º', '9999', 9999, 'm', 24, 'C', '', 'VFR')
Insertada
('GCLA 101000Z 01015KT 9999 BKN030 25/20 Q1017', '2020-08-10T10:00:00Z', 'GCLA', 'Santa Cruz De La Palma', 1017, 'hPa', ['BKN030'], 10, 'º', 15, 'kt', None, None, None, None, '9999', 9999, 'm', 25, 'C', '', 'MVFR')
Insertada
('GCLP 101000Z 36024KT 9999 SCT020 BKN025 24/18 Q1015 NOSIG', '2020-08-10T10:00:00Z', 'GCLP', 'Gran Canaria', 1015, 'hPa', ['SCT020', 'BKN025'], 360, 'º', 24, 'kt', None, None, None, None, '9999', 9999, 'm', 24, 'C', 'NOSIG', 'MVFR')
Insertada
('GCRR 101000Z 01018KT 340V050 9999 FEW024 26/18 Q1014', '2020-08-10T10:00:00Z', 'GCRR

In [60]:
conn.close()